In [1]:
import sys
import subprocess
sys.path.append('f:/seismic/code/core')
sys.path.append('/seismic/code/core')
sys.path.append('/kaggle/input/my-seismic-library/')
import kaggle_support as kgs
kgs.preallocate_matrices=False
import importlib
import seis_model
import time
import pandas as pd
import numpy as np
#kgs.cache_dir_write = kgs.cache_dir_write+'/../cache_evaluate/'
#kgs.cache_dir_read = kgs.cache_dir_write

F:/seismic/models/brendan/*.pth
Loading:  F:/seismic/models/brendan\unet2d_caformer_seed3_epochbest.pt
n_models: 1


In [2]:
# Set up model
importlib.reload(seis_model)
model = seis_model.default_model()
model.read_cache = False

In [3]:
%%time
data = kgs.load_all_train_data(validation_only=True)[::100]
model.models[1].run_in_parallel = True
data_out = model.infer(data)

Importing cache brendan: 100%|██████████████████████████████████████████████████████| 100/100 [00:00<00:00, 390.81it/s]


CPU times: total: 1.19 s
Wall time: 1.47 s


In [4]:
dfTimings = pd.read_csv(kgs.timing_filename)
dfTimings = dfTimings.groupby('family')['time_taken'].mean().to_frame('time_taken')

In [5]:
#kgs.disable_caching = True
families = list(set([d.family for d in data]))
families.sort()
res = dict()
res['family'], res['score_brendan'], res['score_mine'], res['score_mine_max'], res['time_taken'] = [],[],[],[],[]
for f in families:
    res['family'].append(f)
    this_data_out = []
    for d in data_out:
        if d.family==f:
            this_data_out.append(d)
    data_out_brendan = model.models[0].infer(this_data_out)
    res['score_brendan'].append(kgs.score_metric(data_out_brendan,show_diagnostics=False)[0])
    #data_out = model.infer(this_data[:1])
    t=time.time()
    try:
        res['time_taken'].append(dfTimings.loc[f].to_numpy()[0])
    except:
        res['time_taken'].append(0)
    score,_,res_all = kgs.score_metric(this_data_out,show_diagnostics=False)
    res['score_mine'].append(score)
    res['score_mine_max'].append(np.max(res_all))
    #print(pd.DataFrame(res))
df = pd.DataFrame(res)
print('')
print('')
print('Brendan local score: ', np.mean(df['score_brendan']))
print('My local score: ', np.mean(df['score_mine']))
print('Estimate public LB: ', 28.8*np.mean(df['score_mine'])/np.mean(df['score_brendan']))
df

Importing cache brendan: 100%|███████████████████████████████████████████████████████| 10/10 [00:00<00:00, 5000.36it/s]



Brendan local score:  17.173527182567362
My local score:  17.173527182567362
Estimate public LB:  28.8


,family,score_brendan,score_mine,score_mine_max,time_taken
0,CurveFault_A,3.743507,3.743507,8.082754,3.370922
1,CurveFault_B,53.364347,53.364347,143.542449,0.346737
2,CurveVel_A,6.509816,6.509816,15.502449,0.382498
3,CurveVel_B,18.609082,18.609082,40.087755,0.419320
4,FlatFault_A,2.204292,2.204292,4.524144,0.386248
5,FlatFault_B,23.284143,23.284143,105.244286,13.439725
6,FlatVel_A,0.995898,0.995898,2.046122,0.000000
7,FlatVel_B,2.765469,2.765469,8.387755,0.000000
8,Style_A,18.199848,18.199848,44.437395,0.000000
9,Style_B,42.058870,42.058870,65.153283,0.000000


In [6]:
# No extra model: 28.8
# FlatVel:
# StyleA: 

In [7]:
28-30*0.081

25.57